# New Section

In [1]:

import sys
import os

import numpy as np
import pandas as pd


import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder




from tqdm import tqdm
import random 
import datetime

import joblib
import pickle
import tempfile


import random 
import datetime


'''
#LOCAL
ROOT_DIR = os.path.expanduser('~')

folder = ROOT_DIR + '/Downloads/m5-forecasting-accuracy/'

calendar = pd.read_csv(folder+'calendar.csv')
sales_train_evaluation = pd.read_csv(folder+'sales_train_evaluation.csv')
sales_train_validation = pd.read_csv(folder+'sales_train_validation.csv')
sample_submission = pd.read_csv(folder+'sample_submission.csv')
sell_prices = pd.read_csv(folder+'sell_prices.csv')
'''


#'''
#SAGEMAKER:
#!conda install -c conda-forge lightgbm

import boto3
import sagemaker
from sagemaker import get_execution_role


# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


# S3 bucket name
bucket = 'm5-forecasting-accuracy'
subfolder = ''
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']

calendar = pd.read_csv('s3://{}/{}'.format(bucket, 'calendar.csv'))
sales_train_evaluation = pd.read_csv('s3://{}/{}'.format(bucket, 'sales_train_evaluation.csv'))
sales_train_validation = pd.read_csv('s3://{}/{}'.format(bucket, 'sales_train_validation.csv'))
sample_submission = pd.read_csv('s3://{}/{}'.format(bucket, 'sample_submission.csv'))
sell_prices = pd.read_csv('s3://{}/{}'.format(bucket, 'sell_prices.csv'))
#'''
#df_melt = pd.read_pickle('s3://{}/{}'.format(bucket, 'df_melt.pkl'))


**Load the data**

In [2]:
df = sales_train_evaluation


In [3]:
df.head()


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [4]:
calander= calendar 

In [70]:
calendar = None

**data preprocessing**

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
df = reduce_mem_usage(df)
calander = reduce_mem_usage(calander)


Mem. usage decreased to 96.13 Mb (78.8% reduction)
Mem. usage decreased to  0.12 Mb (41.9% reduction)


In [7]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [8]:
calander

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [9]:
c = calander[['date','wm_yr_wk','d']]
df_test = df[:10]
features = ['item_id','cat_id','store_id','state_id','wday','month','year']
cat = ['item_id','cat_id','store_id','state_id']
numcols = [f"d_{day}" for day in range(1,1914)]
df_test = pd.melt(df_test, id_vars=cat, value_vars = numcols, var_name='d',value_name='sales')
df_test = df_test.merge(c, on='d').merge(sell_prices, on=['store_id','item_id','wm_yr_wk'])


In [10]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit_transform(df_test)
df_test


,item_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,sell_price
0,HOBBIES_1_008,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,0.46
1,HOBBIES_1_008,HOBBIES,CA_1,CA,d_2,15,2011-01-30,11101,0.46
2,HOBBIES_1_008,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,0.46
3,HOBBIES_1_008,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,0.46
4,HOBBIES_1_008,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,0.46
...,...,...,...,...,...,...,...,...,...
15891,HOBBIES_1_008,HOBBIES,CA_1,CA,d_1913,1,2016-04-24,11613,0.48
15892,HOBBIES_1_009,HOBBIES,CA_1,CA,d_1912,0,2016-04-23,11613,1.77
15893,HOBBIES_1_009,HOBBIES,CA_1,CA,d_1913,0,2016-04-24,11613,1.77
15894,HOBBIES_1_010,HOBBIES,CA_1,CA,d_1912,0,2016-04-23,11613,2.97


In [11]:
sell_prices.head()


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [12]:
df.head()


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [13]:
ds=2011
de=2016

calander = calander[(calander['year'] >= ds) & (calander['year'] <de)]
# Before one row represents all sales for one item (30490 rows)
# After one row represents one day's sales for one item (58327370 rows = 30490 * 1913)
numcols = [f"d_{day}" for day in range(1,1914)]

In [14]:


# Convert category columns to numbers
# removing dept_id and store from features, not enough training data atm for these
features = ['item_id','cat_id','store_id','state_id','wday','month','year']
cat = ['item_id','cat_id','store_id','state_id']
target = 'sales'
for c in cat:
    df[c] = df[c].astype('category').cat.codes


In [15]:
%time
df = pd.melt(df, id_vars=cat, value_vars = numcols, var_name='d',value_name='sales')
df

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs


,item_id,cat_id,store_id,state_id,d,sales
0,1437,1,0,0,d_1,0
1,1438,1,0,0,d_1,0
2,1439,1,0,0,d_1,0
3,1440,1,0,0,d_1,0
4,1441,1,0,0,d_1,0
...,...,...,...,...,...,...
58327365,1432,0,9,2,d_1913,1
58327366,1433,0,9,2,d_1913,0
58327367,1434,0,9,2,d_1913,0
58327368,1435,0,9,2,d_1913,3


In [17]:
df['d']

0              d_1
1              d_1
2              d_1
3              d_1
4              d_1
             ...  
58327365    d_1913
58327366    d_1913
58327367    d_1913
58327368    d_1913
58327369    d_1913
Name: d, Length: 58327370, dtype: object

In [16]:
calander['d']

0          d_1
1          d_2
2          d_3
3          d_4
4          d_5
         ...  
1793    d_1794
1794    d_1795
1795    d_1796
1796    d_1797
1797    d_1798
Name: d, Length: 1798, dtype: object

In [18]:
df['d'] =  df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

In [19]:
calander['d'] =  calander['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [20]:
%time
df_train = df.merge(calander,on='d')
df_train

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.15 µs


,item_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,1437,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,1438,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,1439,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,1440,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,1441,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54821015,1432,0,9,2,1798,1,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
54821016,1433,0,9,2,1798,0,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
54821017,1434,0,9,2,1798,1,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
54821018,1435,0,9,2,1798,0,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0


In [21]:
df_train = reduce_mem_usage(df_train)

Mem. usage decreased to 3868.82 Mb (0.0% reduction)


In [22]:
df_train

,item_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,1437,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,1438,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,1439,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,1440,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,1441,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54821015,1432,0,9,2,1798,1,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
54821016,1433,0,9,2,1798,0,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
54821017,1434,0,9,2,1798,1,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
54821018,1435,0,9,2,1798,0,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0


In [46]:
data_dir = '../data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
df_train.to_csv(os.path.join(data_dir, 'df_train.csv'), index=False)


In [59]:
df_train_location = session.upload_data(os.path.join(data_dir, 'df_train.csv'))

In [63]:
#boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'df_train.csv')).upload_file(df_train_location)

In [ ]:
df_train = pd.read_csv('s3://{}/{}'.format(bucket, 'df_train.csv'))

In [47]:
df_train

,item_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,1437,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,1438,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,1439,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,1440,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,1441,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54821015,1432,0,9,2,1798,1,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
54821016,1433,0,9,2,1798,0,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
54821017,1434,0,9,2,1798,1,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
54821018,1435,0,9,2,1798,0,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0


In [48]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54821020 entries, 0 to 54821019
Data columns (total 19 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   item_id       int16 
 1   cat_id        int8  
 2   store_id      int8  
 3   state_id      int8  
 4   d             int16 
 5   sales         int16 
 6   date          object
 7   wm_yr_wk      int16 
 8   weekday       object
 9   wday          int8  
 10  month         int8  
 11  year          int16 
 12  event_name_1  object
 13  event_type_1  object
 14  event_name_2  object
 15  event_type_2  object
 16  snap_CA       int8  
 17  snap_TX       int8  
 18  snap_WI       int8  
dtypes: int16(5), int8(8), object(6)
memory usage: 3.8+ GB


In [49]:
df_train.memory_usage(index=True).sum()

4056755480

In [50]:
sys.getsizeof(df_train)

15944288174

In [71]:
calendar = None
sales_train_evaluation = None
sales_train_validation = None
sample_submission = None
sell_prices = None


In [ ]:
df_train[df_train.d < df_train['d'].unique()[-28]]

In [79]:
df_train.d

0              1
1              1
2              1
3              1
4              1
            ... 
54821015    1798
54821016    1798
54821017    1798
54821018    1798
54821019    1798
Name: d, Length: 54821020, dtype: int16

In [80]:
train_df = df_train[df_train.d < df_train['d'].unique()[-28]]

MemoryError: Unable to allocate 418. MiB for an array with shape (54821020,) and data type int64

In [ ]:
val_df= df_train[df_train.d >= df_train['d'].unique()[-28]]

In [ ]:
X = train_df[features].values
X_val = val_df[features].values
y=train_df[target].values
y_val=val_df[target].values


In [ ]:
#paramaters = {'objective':'rmse','metric':'rmse','num_leaves':31}
#model = lgb.train(paramaters,train_data,valid_sets = val_data,num_boost_round = 1000, early_stopping_rounds=100)


In [ ]:
# Creates test data for model.
# ds {int} Date start, inclusive. e.g 1914 (for this comp)
# de {int} Date end, inclusive. e.g 1941 (for this comp)
#
def create_test_data(ds=1914,de=1941):
    calander = pd.read_csv('data/calendar.csv')
    df=pd.read_csv('data/sales_train_validation.csv')
    sell_prices = pd.read_csv('data/sell_prices.csv')
    features = ['item_id','cat_id','state_id','wday','month','year']
    df_features = ['item_id','cat_id','state_id']
    cal_features = ['wday','month','year']
    test_ids = df['id'].unique()
    df_test = df[df_features]
    
    cat = ['item_id','cat_id','state_id']
    for c in cat:
        df_test[c] = df_test[c].astype('category').cat.codes
    
    #test_day_columns=['F'+str(i) for i in range(1,28)]
    calander['d'] =  pd.to_numeric(calander['d'].str.split('_',expand=True)[1])
    calander = calander[(calander.d >= ds) & (calander.d <=de)]
    calander['key'] = 0
    df_test['key'] = 0
    merged = df_test.merge(calander, on='key')
    df_test = merged[features]
    return df_test, df['id']


In [ ]:
def create_28_day_test_submission(test_df,ids,model):
    y_pred = model.predict(test_df.values)
    y_grp = [y_pred[i:i+28] for i  in range(0, len(y_pred), 28)]
    test_day_columns=['F'+str(i) for i in range(1,29)]
    df_sub = pd.DataFrame(columns=test_day_columns,data=y_grp)
    df_sub['id'] = ids
    return df_sub[['id',*test_day_columns]]
    


In [ ]:
ds=1914
de=1941

val_test_data,val_ids = create_test_data(ds,de,calander,df,sell_prices)
